<a href="https://colab.research.google.com/github/msaif28/GenAI/blob/main/Custom_Website_Chatbot_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://ineuron.ai/sitemap.xml

In [19]:
!pip -q install langchain
!pip -q install bitsandbytes accelerate transformers
!pip -q install sentence-transformers
!pip -q install "ipywidgets>=7,<8"
!pip -q install huggingface_hub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [2]:
!pip -q install unstructured
!pip -q install pinecone-client==2.2.4

In [3]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
import pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import torch
from tqdm.autonotebook import tqdm

/home/saifm/demo/notebookenv/lib/python3.8/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Extract data from url

In [4]:
!pip -q install numpy==1.24.4

In [5]:
URLs = [
    "https://ineuron.ai/"
]

In [6]:
loader = UnstructuredURLLoader(urls = URLs)
data = loader.load()

In [7]:
data

[Document(page_content='Learning with iNeuron made \xa0<>\n\nTake your career to the next level with industry ready programs,\n\nAn entire learning ecosystem at your fingertips to make learning fun.\n\nChoose from a range of tech programs and make your next big career switch.\n\nExplore Courses\n\n55%\n\nAverage Salary Hike\n\n400+\n\nDifferent Courses\n\n10000+\n\nCareer Transitions\n\n400+\n\nHiring Partners\n\nLIVE NOW\n\nSupport System\n\nOur support system is live again, this time it is bigger, better and faster.\n\nExperience a tech community like never seen before\n\nTake me there\n\nOur Courses\n\nView all\n\nView all\n\nSuccess Stories\n\nView all\n\nFresher\n\nAbhisekh Bhuyan\n\nMLOps engineer\n\nI got job as an MLOps engineer at synapsica at 13 LPA PPO because of "End to End projects MLOps" from iNeuron.\n\nFrom\n\nFresher\n\nTo\n\n79% Increment\n\nSubham Kanungo\n\nAssociate Data Scientist\n\nI just joined EY as data analyst. It would not be possible with the support of Kri

## Chunkins

In [8]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [9]:
text_chunks=text_splitter.split_documents(data)

In [10]:
len(text_chunks)

6

In [11]:
text_chunks[0]

Document(page_content='Learning with iNeuron made \xa0<>\nTake your career to the next level with industry ready programs,\nAn entire learning ecosystem at your fingertips to make learning fun.\nChoose from a range of tech programs and make your next big career switch.\nExplore Courses\n55%\nAverage Salary Hike\n400+\nDifferent Courses\n10000+\nCareer Transitions\n400+\nHiring Partners\nLIVE NOW\nSupport System\nOur support system is live again, this time it is bigger, better and faster.\nExperience a tech community like never seen before\nTake me there\nOur Courses\nView all\nView all\nSuccess Stories\nView all\nFresher\nAbhisekh Bhuyan\nMLOps engineer\nI got job as an MLOps engineer at synapsica at 13 LPA PPO because of "End to End projects MLOps" from iNeuron.\nFrom\nFresher\nTo\n79% Increment\nSubham Kanungo\nAssociate Data Scientist\nI just joined EY as data analyst. It would not be possible with the support of Krish sir and Sudhanshu sir.\nFrom\nTo\n100% Increment\nSayan Saha\nSo

## Embedding model

In [12]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [13]:
query_result = embeddings.embed_query("Hello world") #embed query is showing 384 because pinecone text dimension size
len(query_result)

384

In [14]:
query_result

[-0.03447727486491203,
 0.031023185700178146,
 0.006734976079314947,
 0.026108963415026665,
 -0.039361998438835144,
 -0.16030248999595642,
 0.06692397594451904,
 -0.006441421806812286,
 -0.04745053872466087,
 0.014758860692381859,
 0.07087533921003342,
 0.05552748590707779,
 0.019193319603800774,
 -0.026251329109072685,
 -0.01010950282216072,
 -0.02694052830338478,
 0.02230740897357464,
 -0.022226683795452118,
 -0.1496926248073578,
 -0.017493069171905518,
 0.007676247972995043,
 0.054352328181266785,
 0.0032544571440666914,
 0.03172598406672478,
 -0.08462140709161758,
 -0.02940600924193859,
 0.05159571394324303,
 0.048124074935913086,
 -0.003314804984256625,
 -0.05827919393777847,
 0.04196930676698685,
 0.022210728377103806,
 0.12818880379199982,
 -0.022338958457112312,
 -0.011656232178211212,
 0.06292840093374252,
 -0.03287631273269653,
 -0.09122604876756668,
 -0.03117530420422554,
 0.05269956961274147,
 0.04703482985496521,
 -0.08420304954051971,
 -0.030056210234761238,
 -0.020744835

In [15]:
import os
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '02ee628c-ec41-4ab8-8f36-0a022f6649ab')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [16]:
import pinecone
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "website-bot" # put in the name of your pinecone index here


In [17]:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

## Create LLM Wrapper

In [20]:
notebook_login()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
tokenizer = AutoTokenizer.from_pretrained("daryl149/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained("daryl149/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              load_in_8bit=True,
                                              #load_in_4bit=True
                                             )

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

/home/saifm/demo/notebookenv/lib/python3.8/site-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


...

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [24]:
pipe = transformers.pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

2024-03-24 15:56:42.080755: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 15:56:42.099550: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-24 15:56:42.447558: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [25]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [26]:
llm.predict("Please provide a concise summary of the Book Harry Potter")

/home/saifm/demo/notebookenv/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


"Please provide a concise summary of the Book Harry Potter and the Philosopher's Stone.\nThis book is a fantasy novel that follows the story of Harry Potter, an orphan who discovers he is a wizard and begins attending Hogwarts School of Witchcraft and Wizardry. The book introduces us to a magical world of wizards, ghosts, and mythical creatures, while following Harry and his friends Ron and Hermione as they uncover a dark plot by the evil wizard Lord Voldemort to steal the powerful Philosopher's Stone. Along the way, they learn magic and make new friends, while also facing challenges and dangers from the dark forces that threaten their world. The book provides a captivating and immersive experience, full of action, adventure, and humor, and has become a beloved classic in the genre of fantasy literature."

## Initialize the Retrieval QA

In [27]:
from langchain.chains import RetrievalQA

In [28]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [31]:
query = "Tell me about the ineuron"

In [32]:
print(qa.run(query))

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

I want to express my gratitude to Krish Naik, Sudhanshu Kumar and Sunny Savita for their instrumental role in my data science journey.
Abhinav Dwivedi
Data Scientist, Softsensor.ai
Thanks to the iNeuron team for teaching me the skillsets to get this fantastic job. it was an amazing journey of 8-9 months. Special thanks to Sudhanshu sir
Saransh Sehrawat
Associate Consultant, EY
A big thanks to the iNeuron team for helping us to enhance our technical knowledge to achieve everyone's dream. My gratitude to all instructors for all you have.
Payel Tarafder
System Engineer, TCS
It was through Krish's videos that I first learned about INEURON, and without hesitation, I joined the inaugural FULL STACK DATA ANALYTICS Course in June 2022.
Rupam Gupta
Business Analyst, Keseya
Hackathon
Embark on an adventure of creativity, teamwork, and